In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [53]:
# 예찬 _ nltk에 저장되어 있는 stop word를 받기 위한 것들
import nltk
from nltk import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [54]:
# 텍스트 정규화
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [55]:
data = pd.read_csv('test.csv', engine ='python') # csv파일 올려서 사용
data.head(1)

,학수번호,과목명,수업년도,수업학기,인문사회,자연과학,공학,예체능,국문개요,영문개요,키워드,추천강의
0,ACC2051,중급회계1,2020.0,1.0,267.0,0.0,3.0,5.0,ACC 251 재무회계 기업 재무회계의 이론적 구조를 이해하고 일반적으로 인정된 회...,Acc 251 Financial Accounting This course is in...,"회계, 재무회계, 회계정보, 의사, 신용의사","회계정보의활용, 기업윤리와사회적책임, 회계정보의이해, 스크린영어, 보이스앙상블"


In [58]:
idx_nm_1 = data[data['영문개요'].isnull() == True].index
data = data.drop(idx_nm_1)
data['newidx'] = range(0,len(data))
data = data.set_index("newidx") # newidx를 df의 새로운 인덱스로 만듦
data

,학수번호,과목명,수업년도,수업학기,인문사회,자연과학,공학,예체능,국문개요,영문개요,키워드,추천강의
newidx,,,,,,,,,,,,
0,ACC2051,중급회계1,2020.0,1.0,267.0,0.0,3.0,5.0,ACC 251 재무회계 기업 재무회계의 이론적 구조를 이해하고 일반적으로 인정된 회...,Acc 251 Financial Accounting This course is in...,"회계, 재무회계, 회계정보, 의사, 신용의사","회계정보의활용, 기업윤리와사회적책임, 회계정보의이해, 스크린영어, 보이스앙상블"
1,AIX0002,AI+X:인공지능,2020.0,2.0,4.0,1.0,10.0,0.0,본 교과목은 4차산업혁명의 핵심기술인 인공지능의 개념과 원리를 이해하고 적용할 수 ...,artificial intelligence,"학문단위, 인공지능, 지능, 교과목, 학문","AI+X:머신러닝, 영화속심리학, 사이버토익2, 특수교육학개론, AI+X:R-Py컴퓨팅"
2,AIX0003,AI+X:딥러닝,2020.0,2.0,2.0,1.0,24.0,1.0,"본 교과목은 딥러닝을 개념을 이해하고, 딥러닝을 실험해 볼 수 있는 개발환경 조성과...",Introduction to Deep Learning,"딥러닝, 인공지능, 텔레콤, 동영상, 발표자료",NaN
3,AIX0004,AI+X:R-Py컴퓨팅,2020.0,2.0,5.0,6.0,17.0,0.0,본 교과목은 데이터 사이언스 및 AI 과목에 기초가 되는 R 프로그래밍 및 Pyth...,R-Py computing,"데이터, 프로그래밍, 기초, 머신러닝, 반복문과",NaN
4,AIX0005,AI+X:머신러닝,2020.0,2.0,0.0,1.0,28.0,0.0,본 교과목은 머신러닝의 이해를 돕기 위한 사례 예제 중심으로 학습한다. 머신러닝의 ...,Machine Learning,"머신러닝, 이해, 러닝, 머신, 딥러닝",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
209,LIT0013,학기제현장실습(전공핵심),2020.0,2.0,28.0,3.0,43.0,7.0,"대학-산업체간 인력양성, skill mismatch 문제를 해결하는 방안 중 하나가...",ERICA Work Placement is an educational curricu...,NaN,NaN
210,LIT0014,학기제현장실습(전공심화),2020.0,2.0,29.0,2.0,50.0,7.0,"대학-산업체간 인력양성, skill mismatch 문제를 해결하는 방안 중 하나가...",ERICA Work Placement is an educational curricu...,NaN,NaN
211,SYB9031,문화기술과사회변동,2020.0,2.0,58.0,2.0,23.0,10.0,문화와 기술의 결합을 의미하는 CT(Culture Technology)는 문화콘텐츠...,"· CT (Culture Technology), which means a combi...",NaN,NaN


In [48]:
# 채린: eda_nlp data augmentation 추가
# EDA: Easy Data Augmentation Techniques for Boosting Performance on Text Classification Tasks 논문 구현 참고
import random
from random import shuffle
random.seed(1)

# 채린_ stop words 지정
stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 
			'ours', 'ourselves', 'you', 'your', 'yours', 
			'yourself', 'yourselves', 'he', 'him', 'his', 
			'himself', 'she', 'her', 'hers', 'herself', 
			'it', 'its', 'itself', 'they', 'them', 'their', 
			'theirs', 'themselves', 'what', 'which', 'who', 
			'whom', 'this', 'that', 'these', 'those', 'am', 
			'is', 'are', 'was', 'were', 'be', 'been', 'being', 
			'have', 'has', 'had', 'having', 'do', 'does', 'did',
			'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or',
			'because', 'as', 'until', 'while', 'of', 'at', 
			'by', 'for', 'with', 'about', 'against', 'between',
			'into', 'through', 'during', 'before', 'after', 
			'above', 'below', 'to', 'from', 'up', 'down', 'in',
			'out', 'on', 'off', 'over', 'under', 'again', 
			'further', 'then', 'once', 'here', 'there', 'when', 
			'where', 'why', 'how', 'all', 'any', 'both', 'each', 
			'few', 'more', 'most', 'other', 'some', 'such', 'no', 
			'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 
			'very', 's', 't', 'can', 'will', 'just', 'don', 
			'should', 'now', '']

# 채린_ 텍스트 클렌징
import re
def get_only_chars(line):

    clean_line = ""

    line = line.replace("’", "")
    line = line.replace("'", "")
    line = line.replace("-", " ") #replace hyphens with spaces
    line = line.replace("\t", " ")
    line = line.replace("\n", " ")
    line = line.lower()

    for char in line:
        if char in 'qwertyuiopasdfghjklzxcvbnm ':
            clean_line += char
        else:
            clean_line += ' '

    clean_line = re.sub(' +',' ',clean_line) #delete extra spaces
    if clean_line[0] == ' ':
        clean_line = clean_line[1:]
    return clean_line



# 채린_ 문장의 단어를 wordnet의 동의어로 교체
#import nltk
#nltk.download('wordnet')
from nltk.corpus import wordnet 

def synonym_replacement(words, n):
	new_words = words.copy()
	random_word_list = list(set([word for word in words if word not in stop_words]))
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			#print("replaced", random_word, "with", synonym)
			num_replaced += 1
		if num_replaced >= n: # 채린_ 최대 n 개의 단어만 교체
			break

	sentence = ' '.join(new_words)
	new_words = sentence.split(' ')

	return new_words

def get_synonyms(word):
	synonyms = set()
	for syn in wordnet.synsets(word): 
		for l in syn.lemmas(): 
			synonym = l.name().replace("_", " ").replace("-", " ").lower()
			synonym = "".join([char for char in synonym if char in ' qwertyuiopasdfghjklzxcvbnm'])
			synonyms.add(synonym) 
	if word in synonyms:
		synonyms.remove(word)
	return list(synonyms)


# 채린_ 무작위 삭제. 확률 P로 문장에서 단어를 무작위로 삭제
def random_deletion(words, p):

	# 채린_ 한 단어만 있으면 지우지 않는다
	if len(words) == 1:
		return words

	# 채린_ 확률이 p 인 단어를 무작위로 삭제
	new_words = []
	for word in words:
		r = random.uniform(0, 1)
		if r > p:
			new_words.append(word)

	#if you end up deleting all words, just return a random word
	if len(new_words) == 0:
		rand_int = random.randint(0, len(words)-1)
		return [words[rand_int]]

	return new_words




# 채린_ 문장에서 두 단어를 무작위로 n 번 교환
def random_swap(words, n):
	new_words = words.copy()
	for _ in range(n):
		new_words = swap_word(new_words)
	return new_words

def swap_word(new_words):
	random_idx_1 = random.randint(0, len(new_words)-1)
	random_idx_2 = random_idx_1
	counter = 0
	while random_idx_2 == random_idx_1:
		random_idx_2 = random.randint(0, len(new_words)-1)
		counter += 1
		if counter > 3:
			return new_words
	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1] 
	return new_words



# 문장에 n 개의 단어를 무작위로 삽입
def random_insertion(words, n):
	new_words = words.copy()
	for _ in range(n):
		add_word(new_words)
	return new_words

def add_word(new_words):
	synonyms = []
	counter = 0
	while len(synonyms) < 1:
		random_word = new_words[random.randint(0, len(new_words)-1)]
		synonyms = get_synonyms(random_word)
		counter += 1
		if counter >= 10:
			return
	random_synonym = synonyms[0]
	random_idx = random.randint(0, len(new_words)-1)
	new_words.insert(random_idx, random_synonym)




# 채린_ data augmentation 핵심함수!!
def eda(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):
	
	sentence = get_only_chars(sentence)
	words = sentence.split(' ')
	words = [word for word in words if word is not '']
	num_words = len(words)
	
	augmented_sentences = []
	num_new_per_technique = int(num_aug/4)+1

	#sr
	if (alpha_sr > 0):
		n_sr = max(1, int(alpha_sr*num_words))
		for _ in range(num_new_per_technique):
			a_words = synonym_replacement(words, n_sr)
			augmented_sentences.append(' '.join(a_words))

	#ri
	if (alpha_ri > 0):
		n_ri = max(1, int(alpha_ri*num_words))
		for _ in range(num_new_per_technique):
			a_words = random_insertion(words, n_ri)
			augmented_sentences.append(' '.join(a_words))

	#rs
	if (alpha_rs > 0):
		n_rs = max(1, int(alpha_rs*num_words))
		for _ in range(num_new_per_technique):
			a_words = random_swap(words, n_rs)
			augmented_sentences.append(' '.join(a_words))

	#rd
	if (p_rd > 0):
		for _ in range(num_new_per_technique):
			a_words = random_deletion(words, p_rd)
			augmented_sentences.append(' '.join(a_words))

	augmented_sentences = [get_only_chars(sentence) for sentence in augmented_sentences]
	shuffle(augmented_sentences)

	#채린_ 원하는 수의 증강 문장을 갖도록 트리밍
	if num_aug >= 1:
		augmented_sentences = augmented_sentences[:num_aug]
	else:
		keep_prob = num_aug / len(augmented_sentences)
		augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]

	# 채린_ 원문 추가
	augmented_sentences.append(sentence)

	return augmented_sentences

In [61]:
data['영문개요'][25]

'this run is suitable for the students who have not learned computer aided design softwares in the class students learn the official rules and regulations of the design by practicing the actual get universal introduction to computer aided design and its application to industrial design produce design draftsmanship presets and user settings of certain application and authoring design specifications will be discussed.this course is design for the students suitable and actual learned computer aided softwares design in the class students will the official rules and regulations of the design by practicing the not drawings general introduction to computer aided design and its application to industrial design creating who drafting presets and user settings of certain applications have authoring design specifications learn be discussed.this course is suitable for the students who have not learned computer aided design softwares in the students learn the official rules and regulations of the de

In [60]:
for i in range(len(data)):
  data['영문개요'][i] = '.'.join(eda(str(data['영문개요'][i])))

data['영문개요'][0]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


'acc financial accounting this course is intermediate accounting it covers basic theories conceptual frameworks financial statements involving assets liabilities and equity this course improves ability for to make and credit decision to analyze future cash flow in business.acc financial accounting this naturally is intermediate accounting it covers basic theories conceptual frameworks of financial statements involving assets liabilities and owners equity this naturally improves ability for students to take in investment and credit decision to analyze future john cash menstruation in business.acc financial ability this course is intermediate accounting it this business theories conceptual frameworks of future statements involving assets liabilities and owners equity covers course improves accounting for students to make investment and credit decision to analyze financial cash flow in basic.acc financial accounting this course is decisiveness intermediate accounting it hereafter covers l

In [62]:
tfidf = TfidfVectorizer(stop_words='english')
# 국문개요에 대해서 tf-idf 수행
tfidf_matrix = tfidf.fit_transform(data['영문개요'])
print(tfidf_matrix.shape)

(214, 4193)


In [63]:
# 예찬 _ 영문 개요를 word_tokenize를 통해서 문장을 각 단어들로 쪼개서 저장함
words = word_tokenize(data['영문개요'][0])
print(words)

['acc', 'financial', 'accounting', 'this', 'course', 'is', 'intermediate', 'accounting', 'it', 'covers', 'basic', 'theories', 'conceptual', 'frameworks', 'financial', 'statements', 'involving', 'assets', 'liabilities', 'and', 'equity', 'this', 'course', 'improves', 'ability', 'for', 'to', 'make', 'and', 'credit', 'decision', 'to', 'analyze', 'future', 'cash', 'flow', 'in', 'business.acc', 'financial', 'accounting', 'this', 'naturally', 'is', 'intermediate', 'accounting', 'it', 'covers', 'basic', 'theories', 'conceptual', 'frameworks', 'of', 'financial', 'statements', 'involving', 'assets', 'liabilities', 'and', 'owners', 'equity', 'this', 'naturally', 'improves', 'ability', 'for', 'students', 'to', 'take', 'in', 'investment', 'and', 'credit', 'decision', 'to', 'analyze', 'future', 'john', 'cash', 'menstruation', 'in', 'business.acc', 'financial', 'ability', 'this', 'course', 'is', 'intermediate', 'accounting', 'it', 'this', 'business', 'theories', 'conceptual', 'frameworks', 'of', 'fut

In [64]:
filtered_words = []
# 예찬 _ nltk에 있는 stopword를 필터해서 새로 저장함
stopwords = nltk.corpus.stopwords.words('english')

for word in words:
    if word not in stopwords:
        # 예찬 _ 새로운 filtered_words에 filter를 마친 단어들을 저장함
        filtered_words.append(word)
        
print(filtered_words)

['acc', 'financial', 'accounting', 'course', 'intermediate', 'accounting', 'covers', 'basic', 'theories', 'conceptual', 'frameworks', 'financial', 'statements', 'involving', 'assets', 'liabilities', 'equity', 'course', 'improves', 'ability', 'make', 'credit', 'decision', 'analyze', 'future', 'cash', 'flow', 'business.acc', 'financial', 'accounting', 'naturally', 'intermediate', 'accounting', 'covers', 'basic', 'theories', 'conceptual', 'frameworks', 'financial', 'statements', 'involving', 'assets', 'liabilities', 'owners', 'equity', 'naturally', 'improves', 'ability', 'students', 'take', 'investment', 'credit', 'decision', 'analyze', 'future', 'john', 'cash', 'menstruation', 'business.acc', 'financial', 'ability', 'course', 'intermediate', 'accounting', 'business', 'theories', 'conceptual', 'frameworks', 'future', 'statements', 'involving', 'assets', 'liabilities', 'owners', 'equity', 'covers', 'course', 'improves', 'accounting', 'students', 'make', 'investment', 'credit', 'decision', 

In [65]:
# 예찬 _ 의도하고 싶은 것 : 전체 영문개요의 내용을 일괄적으로 stop word를 이용해서 필터하고 그 내용을 가지고 코사인 유사도를 구하고 싶다 !
# 채린_전체 영문개요 stop word 필터링 및 결과 문장으로 붙이기
filtered_words = [' ' for _ in range(len(data['국문개요']))]
stopwords = nltk.corpus.stopwords.words('english')
# 예찬 _ 정규화 작업을 위해서 WordNetLemmatizer()호출
lemma = WordNetLemmatizer()
for i in range(len(data['영문개요'])):
  words = word_tokenize(data['영문개요'][i])
  # print(words)
  for word in words:
    if word not in stopwords:
        # 예찬 _ word에 속한 단어를 명사로 변환
        word = lemma.lemmatize(word, 'n')
        # 예찬 _ 새로운 filtered_words에 filter를 마친 단어들을 저장함
        filtered_words[i] += (word+' ')
print(filtered_words)
# print(len(filtered_words))

[' acc financial accounting course intermediate accounting cover basic theory conceptual framework financial statement involving asset liability equity course improves ability make credit decision analyze future cash flow business.acc financial accounting naturally intermediate accounting cover basic theory conceptual framework financial statement involving asset liability owner equity naturally improves ability student take investment credit decision analyze future john cash menstruation business.acc financial ability course intermediate accounting business theory conceptual framework future statement involving asset liability owner equity cover course improves accounting student make investment credit decision analyze financial cash flow basic.acc financial accounting course decisiveness intermediate accounting hereafter cover liaise basic theory conceptual framework financial statement involving asset liability owner equity course improves ability student make investment credit deci

In [ ]:
# import math 
# import numpy as np
# import torch

# def ts_ss_(v, eps=1e-15, eps2=1e-4):
#     # reusable compute
#     v_inner = torch.mm(v, v.t())
#     vs = v.norm(dim=-1, keepdim=True)
#     vs_dot = vs.mm(vs.t())

#     # compute triangle(v)
#     v_cos = v_inner / vs_dot
#     v_cos = v_cos.clamp(-1. + eps2, 1. - eps2)  # clamp to avoid backprop instability
#     theta_ = torch.acos(v_cos) + math.radians(10)
#     theta_rad = theta_ * math.pi / 180.
#     tri = (vs_dot * torch.sin(theta_rad)) / 2.

#     # compute sector(v)
#     v_norm = (v ** 2).sum(-1, keepdim=True)
#     euc_dist = v_norm + v_norm.t() - 2.0 * v_inner
#     euc_dist = torch.sqrt(torch.abs(euc_dist) + eps)  # add epsilon to avoid srt(0.)
#     magnitude_diff = (vs - vs.t()).abs()
#     sec = math.pi * (euc_dist + magnitude_diff) ** 2 * theta_ / 360.

#     return tri * sec

# v = torch.tensor(data['영문개요'])
# print(ts_ss_(v))

In [66]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [67]:
indices = pd.Series(data.index, index=data['과목명']).drop_duplicates()
print(indices.head())

과목명
중급회계1           0
AI+X:인공지능       1
AI+X:딥러닝        2
AI+X:R-Py컴퓨팅    3
AI+X:머신러닝       4
Name: newidx, dtype: int64


In [68]:
# from google.colab import drive
# drive.mount('/content/drive')

In [69]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # 선택한 국문개요에 해당되는 인덱스를 받아옵니다. 이제 선택한 영화를 가지고 연산할 수 있습니다.
    idx = indices[title]

    # 모든 과목에 대해서 해당 과목와의 유사도를 구합니다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 정렬합니다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 과목를 받아옵니다.
    sim_scores = sim_scores[1:6]

    # 가장 유사한 10개의 과목의 인덱스를 받아옵니다.
    movie_indices = [i[0] for i in sim_scores]

    # 가장 유사한 10개의 영화의 제목을 리턴합니다.
    return data['과목명'].iloc[movie_indices]

In [70]:
get_recommendations('실용중국어')

newidx
91          중급중국어
106       한류와한국문화
187      한국의문화와역사
141    공학도를위한학술영어
204        C프로그래밍
Name: 과목명, dtype: object

In [76]:
for i in range(len(data)):
  print(i+1,'번째 과목명: ', indices.index[i])
  print(get_recommendations(indices.index[i]))
  print("==================================")

1 번째 과목명:  중급회계1
newidx
6     회계정보의이해
43    회계정보의활용
11       창의역량
88      경영학원론
9      경영자료분석
Name: 과목명, dtype: object
2 번째 과목명:  AI+X:인공지능
newidx
154     딥러닝의원리와활용
177        인공지능입문
176    융합과함께하는ICT
9          경영자료분석
150        뉴스바로보기
Name: 과목명, dtype: object
3 번째 과목명:  AI+X:딥러닝
newidx
154       딥러닝의원리와활용
4         AI+X:머신러닝
177          인공지능입문
199    아카데믹한국어듣기말하기
190        기억력과학습전략
Name: 과목명, dtype: object
4 번째 과목명:  AI+X:R-Py컴퓨팅
newidx
28     그림으로배우는컴퓨터구조기초(클러스터)
85                  계절제현장실습
210           학기제현장실습(전공심화)
209           학기제현장실습(전공핵심)
207           학기제현장실습(교양선택)
Name: 과목명, dtype: object
5 번째 과목명:  AI+X:머신러닝
newidx
58                  기계비평
177               인공지능입문
2               AI+X:딥러닝
29     멀티미디어정보처리기초(클러스터)
199         아카데믹한국어듣기말하기
Name: 과목명, dtype: object
6 번째 과목명:  경영통계분석1
newidx
160    불확실한세상과통계학
9          경영자료분석
88          경영학원론
0           중급회계1
6         회계정보의이해
Name: 과목명, dtype: object
7 번째 과목명:  회계정보의이해
newidx
43    회계정보의활용
0       중급회계1
88      경영학원론
95 